# Description
This kernel performs inference for [PANDA concat tile pooling starter](https://www.kaggle.com/iafoss/panda-concat-fast-ai-starter) kernel with use of multiple models and 8 fold TTA. Check it for more training details. The image preprocessing pipline is provided [here](https://www.kaggle.com/iafoss/panda-16x128x128-tiles).

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")
import random
import numpy as np
import pandas as pd
import warnings
from pathlib import Path
from typing import Callable, Tuple
import torch.utils.data
import matplotlib.pyplot as plt
import PIL.Image
from torch.utils.data import DataLoader, Dataset
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd


torch.__version__

'2.1.2'

In [3]:
df_test = pd.read_csv("../input/prostate-cancer-grade-assessment/test.csv")
df_test.head()

,image_id,data_provider
0,005700be7e06878e6605e7a5a39de1b2,radboud
1,005c6e8877caf724c600fdce5d417d40,karolinska
2,0104f76634ff89bfff1ef0804a95c380,radboud


In [4]:
img_path = "../input/panda-16x128x128-tiles-data/test"
img_id = list(df_test["image_id"])
img_files = glob.glob(img_path + f"/{img_id[1]}" + "*")

In [5]:
def get_df():
    data = {"image_id": []}
    img_path = "../input/panda-16x128x128-tiles-data/test"
    img_ids = list(df_test["image_id"])
    for i in tqdm(range(len(img_ids))):
        img_id = img_ids[i]
        img_files = []
        for i in range(16):
            img_files.append(f"{img_id}"+f"_{i}"+".png")
        data["image_id"].extend(img_files)
        
        
    return data

In [6]:
def to_pandas(data):
    df = pd.DataFrame(data, columns = ["image_id"])
    return df

In [7]:
data = get_df()
df_new = to_pandas(data)

df_new.head()

  0%|          | 0/3 [00:00<?, ?it/s]

,image_id
0,005700be7e06878e6605e7a5a39de1b2_0.png
1,005700be7e06878e6605e7a5a39de1b2_1.png
2,005700be7e06878e6605e7a5a39de1b2_2.png
3,005700be7e06878e6605e7a5a39de1b2_3.png
4,005700be7e06878e6605e7a5a39de1b2_4.png


In [8]:
def get_img_tiles(image_list):
    img_rows = []
    rc = 0
    for i in range(3):
        img1 = cv2.imread(image_list[rc + 0], cv2.COLOR_BGR2RGB)
        img2 = cv2.imread(image_list[rc + 1], cv2.COLOR_BGR2RGB)
        img3 = cv2.imread(image_list[rc + 2], cv2.COLOR_BGR2RGB)
     
        
        # Check if any image is None
        if any(img is None for img in [img1, img2, img3]):
            print("Some images could not be loaded properly. Skipping concatenation.")
            return "wrong"
        
        img_row = np.concatenate((img1, img2, img3), axis=1)
        rc += 3
        img_rows.append(img_row)
    img_stacked = np.concatenate(img_rows, axis=0)
    return img_stacked

In [9]:
from tqdm import tqdm

In [10]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Define a list to store example tiles
example_tiles_list = []

# Split the DataFrame into chunks of size 16 and process each chunk
for i in tqdm(range(0, len(df_new), 16)):
    example_chunk = df_new["image_id"].iloc[i:i+12]
    example_chunk = list(example_chunk.map(lambda x: os.path.join("../input/panda-16x128x128-tiles-data/test", x)))

    # Concatenate the tiles
    example_tile = get_img_tiles(example_chunk)
    if str(example_tile) == "wrong":
        continue
    example_tiles_list.append(example_tile)
    

print(len(example_tiles_list))

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        if self.transform:
            img = self.transform(img)
        return img

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create the custom dataset
custom_dataset = CustomDataset(example_tiles_list, transform=transform)

test_size = len(custom_dataset)
test_dataset = torch.utils.data.random_split(custom_dataset, [test_size])


  0%|          | 0/3 [00:00<?, ?it/s][ WARN:0@11.534] global loadsave.cpp:248 findDecoder imread_('../input/panda-16x128x128-tiles-data/test/005700be7e06878e6605e7a5a39de1b2_0.png'): can't open/read file: check file path/integrity
[ WARN:0@11.535] global loadsave.cpp:248 findDecoder imread_('../input/panda-16x128x128-tiles-data/test/005700be7e06878e6605e7a5a39de1b2_1.png'): can't open/read file: check file path/integrity
[ WARN:0@11.535] global loadsave.cpp:248 findDecoder imread_('../input/panda-16x128x128-tiles-data/test/005700be7e06878e6605e7a5a39de1b2_2.png'): can't open/read file: check file path/integrity
[ WARN:0@11.535] global loadsave.cpp:248 findDecoder imread_('../input/panda-16x128x128-tiles-data/test/005c6e8877caf724c600fdce5d417d40_0.png'): can't open/read file: check file path/integrity
[ WARN:0@11.536] global loadsave.cpp:248 findDecoder imread_('../input/panda-16x128x128-tiles-data/test/005c6e8877caf724c600fdce5d417d40_1.png'): can't open/read file: check file path/int

Some images could not be loaded properly. Skipping concatenation.
Some images could not be loaded properly. Skipping concatenation.
Some images could not be loaded properly. Skipping concatenation.
0


In [13]:
import timm
from torch.nn import AdaptiveAvgPool2d, Flatten, Linear
from mish_activation import Mish 

class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, output_size=(1, 1)):
        super(AdaptiveConcatPool2d, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(output_size)
        self.max_pool = nn.AdaptiveMaxPool2d(output_size)

    def forward(self, x):
        return torch.cat([self.avg_pool(x), self.max_pool(x)], 1)

class Model(nn.Module):
    def __init__(self, arch='resnext50_32x4d', n=6, pre=True):
        super().__init__()
        # Load ResNet model using timm
        m = timm.create_model(arch, pretrained=pre)
        self.enc = nn.Sequential(*list(m.children())[:-2])
        nc = list(m.children())[-1].in_features
        self.head = nn.Sequential(AdaptiveConcatPool2d(), Flatten(), nn.Linear(2 * nc, 512),
                                  Mish(), nn.BatchNorm1d(512), nn.Dropout(0.5), nn.Linear(512, n))
        
    def forward(self, *x):
        shape = x[0].shape
        n = len(x)
        x = torch.stack(x, 1).view(-1, shape[1], shape[2], shape[3])
        # x: bs*N x 3 x 128 x 128
        x = self.enc(x)
        # x: bs*N x C x 4 x 4
        shape = x.shape
        # concatenate the output for tiles into a single map
        x = x.view(-1, n, shape[1], shape[2], shape[3]).permute(0, 2, 1, 3, 4).contiguous() \
            .view(-1, shape[1], shape[2] * n, shape[3])
        # x: bs x C x N*4 x 4
        x = self.head(x)
        # x: bs x n
        return x

In [14]:
# model = Model()
# pretrained_weights_path = '../input/publicpandasetraul/resnext50_32x4d_final.pth'

# # Load the pre-trained weights into the model
# model.load_state_dict(torch.load(pretrained_weights_path))

# # Ensure the model is in evaluation mode
# model.eval()
print("modelbefore")
model = torch.load('/kaggle/input/modelagain/resnext50_32x4d_final (1).pth')
print("modelafter")

modelbefore
modelafter


In [45]:
pred = []

try:
    for image in test_dataset:
        image = images.to(device)  # Move data to GPU
        output = model(image)
        pred.append(output.cpu())
except:
    pass

# Concatenate predictions and targets
try:
    pred = torch.cat(pred, 0)
    
except:
    pass


# Calculate Kappa score and confusion matrix
try:
    p = torch.argmax(pred, 1)
    pred.append(p)
except:
    p = 0
    pred.append(p)



In [46]:
df_test.drop(columns=['data_provider'], inplace=True)

In [47]:
df_test

,image_id
0,005700be7e06878e6605e7a5a39de1b2
1,005c6e8877caf724c600fdce5d417d40
2,0104f76634ff89bfff1ef0804a95c380


In [48]:
while len(p) < len(df_test):
    p.append(0)  

df_test['isup_grade'] = p

In [49]:
df_test

,image_id,isup_grade
0,005700be7e06878e6605e7a5a39de1b2,0
1,005c6e8877caf724c600fdce5d417d40,0
2,0104f76634ff89bfff1ef0804a95c380,0


In [50]:
df_test.to_csv("submission.csv", index=False)
df_test.head()

,image_id,isup_grade
0,005700be7e06878e6605e7a5a39de1b2,0
1,005c6e8877caf724c600fdce5d417d40,0
2,0104f76634ff89bfff1ef0804a95c380,0


In [52]:
#print(model)